In [1]:
import pandas as pd
import numpy as np
import gensim
import nltk
from nltk.corpus import stopwords
from scipy.stats import skew, kurtosis
import re
from gensim.models import Word2Vec
import gensim
from gensim.models.wrappers import FastText
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import KFold
from gensim.models import KeyedVectors
import pickle
Word2Vec = KeyedVectors.load_word2vec_format('../diff_traditional/source_data/wiki.es.vec', binary=False)

/home/nikuailemateam/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/nikuailemateam/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/nikuailemateam/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/nikuailemateam/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# f_in = open('word2vec','rb')
# Word2Vec = pickle.load(f_in)
train = pd.read_csv('preprocessing_train.csv')
test = pd.read_csv('preprocessing_test.csv')
test['label'] = -1
test['test_id'] = range(len(test))

In [5]:
print ("去掉不在word2vec中的词...")#不去除停用词的原因在于停用词里面有否定词,会对语义产生影响
def spa_norm_word_list(s1):
    s1 = str(s1).lower().split()
    
    return [w for w in s1 if w in Word2Vec or w.isdigit()]

train['spa_qura1_list'] = train['spa_qura1'].apply(spa_norm_word_list)
train['spa_qura2_list'] = train['spa_qura2'].apply(spa_norm_word_list)

test['spa_qura1_list'] = test['spa_qura1'].apply(spa_norm_word_list)
test['spa_qura2_list'] = test['spa_qura2'].apply(spa_norm_word_list)

print("去掉停用词结束...")

去掉不在word2vec中的词...
去掉停用词结束...


In [17]:
train.shape

(21400, 5)

In [18]:
test.shape

(10000, 6)

In [6]:
train.head(30)

,label,spa_qura1,spa_qura2,spa_qura1_list,spa_qura2_list
0,0,hola hago clic en el producto recibido,Compré un producto y no he recibido un correo ...,"[hola, hago, clic, en, el, producto, recibido]","[compré, un, producto, y, no, he, recibido, un..."
1,0,Hola Cerré la disputa el 21 de mayo de 2017 y ...,No obtuve el reembolso de mi dinero Han pasado...,"[hola, cerré, la, disputa, el, 21, de, mayo, d...","[no, obtuve, el, reembolso, de, mi, dinero, ha..."
2,0,Ordené de España a España ahora que mandan el ...,Mi pedido llegó pero el color es diferente al ...,"[ordené, de, españa, a, españa, ahora, que, ma...","[mi, pedido, llegó, pero, el, color, es, difer..."
3,1,Debo pagar impuestos personalizados,Cómo pagar los derechos de aduana,"[debo, pagar, impuestos, personalizados]","[cómo, pagar, los, derechos, de, aduana]"
4,0,No recibí mi pedido,Mi pedido muestra que no he pagado pero lo hice,"[no, recibí, mi, pedido]","[mi, pedido, muestra, que, no, he, pagado, per..."
5,0,Hola Eva Al principio la pregunta Puedo escrib...,Cómo puedo escribir comentarios para los produ...,"[hola, eva, al, principio, la, pregunta, puedo...","[cómo, puedo, escribir, comentarios, para, los..."
6,0,estoy tan enojado,Abrí una disputa y hasta ahora nada,"[estoy, tan, enojado]","[abrí, una, disputa, y, hasta, ahora, nada]"
7,0,No puedo abrir una disputa,No puedo rastrear,"[no, puedo, abrir, una, disputa]","[no, puedo, rastrear]"
8,0,Cómo puedo verificar el número de pedido,Dónde puedo ver el número de mi rastreador de ...,"[cómo, puedo, verificar, el, número, de, pedido]","[dónde, puedo, ver, el, número, de, mi, rastre..."
9,0,No necesito una respuesta tuya,Necesito una factura por este artículo que te ...,"[no, necesito, una, respuesta, tuya]","[necesito, una, factura, por, este, artículo, ..."


In [6]:
train['max_length'] = train.apply(lambda x: max(len(x['spa_qura1_list']),len(x['spa_qura2_list'])),axis = 1)
test['max_length'] = test.apply(lambda x: max(len(x['spa_qura1_list']),len(x['spa_qura2_list'])),axis = 1)

In [7]:
train.head(10)

,label,spa_qura1,spa_qura2,spa_qura1_list,spa_qura2_list,max_length
0,0,hola hago clic en el producto recibido,Compré un producto y no he recibido un correo ...,"[hola, hago, clic, en, el, producto, recibido]","[compré, un, producto, y, no, he, recibido, un...",12
1,0,Hola Cerré la disputa el 21 de mayo de 2017 y ...,No obtuve el reembolso de mi dinero Han pasado...,"[hola, cerré, la, disputa, el, de, mayo, de, y...","[no, obtuve, el, reembolso, de, mi, dinero, ha...",20
2,0,Ordené de España a España ahora que mandan el ...,Mi pedido llegó pero el color es diferente al ...,"[ordené, de, españa, a, españa, ahora, que, ma...","[mi, pedido, llegó, pero, el, color, es, difer...",12
3,1,Debo pagar impuestos personalizados,Cómo pagar los derechos de aduana,"[debo, pagar, impuestos, personalizados]","[cómo, pagar, los, derechos, de, aduana]",6
4,0,No recibí mi pedido,Mi pedido muestra que no he pagado pero lo hice,"[no, recibí, mi, pedido]","[mi, pedido, muestra, que, no, he, pagado, per...",10
5,0,Hola Eva Al principio la pregunta Puedo escrib...,Cómo puedo escribir comentarios para los produ...,"[hola, eva, al, principio, la, pregunta, puedo...","[cómo, puedo, escribir, comentarios, para, los...",10
6,0,estoy tan enojado,Abrí una disputa y hasta ahora nada,"[estoy, tan, enojado]","[abrí, una, disputa, y, hasta, ahora, nada]",7
7,0,No puedo abrir una disputa,No puedo rastrear,"[no, puedo, abrir, una, disputa]","[no, puedo, rastrear]",5
8,0,Cómo puedo verificar el número de pedido,Dónde puedo ver el número de mi rastreador de ...,"[cómo, puedo, verificar, el, número, de, pedido]","[dónde, puedo, ver, el, número, de, mi, rastre...",10
9,0,No necesito una respuesta tuya,Necesito una factura por este artículo que te ...,"[no, necesito, una, respuesta, tuya]","[necesito, una, factura, por, este, artículo, ...",9


In [8]:
len(np.where(train.max_length > 51)[0])

17

In [9]:
len(np.where(test.max_length > 51)[0])

1

In [8]:
print("生成embedding...")

spa_list = list(train['spa_qura1_list'])
spa_list.extend(list(train['spa_qura2_list']))
spa_list.extend(list(test['spa_qura1_list']))
spa_list.extend(list(test['spa_qura2_list']))

MAX_NB_WORDS = 200000
MAX_SEQUENCE_LENGTH = 55
EMBEDDING_DIM = 300

# 生成token词典
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(spa_list)

texts_1 = list(train['spa_qura1_list'])
texts_2 = list(train['spa_qura2_list'])
test_texts_1 = list(test['spa_qura1_list'])
test_texts_2 = list(test['spa_qura2_list'])

# 得到词索引,相当于把每个sentence给序列化
sequences_1 = tokenizer.texts_to_sequences(texts_1)
sequences_2 = tokenizer.texts_to_sequences(texts_2)
test_sequences_1 = tokenizer.texts_to_sequences(test_texts_1)
test_sequences_2 = tokenizer.texts_to_sequences(test_texts_2)

# number of unique words
word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

# 长度不足max_length的补0
data_1 = pad_sequences(sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
data_2 = pad_sequences(sequences_2, maxlen=MAX_SEQUENCE_LENGTH)

test_data_1 = pad_sequences(test_sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
test_data_2 = pad_sequences(test_sequences_2, maxlen=MAX_SEQUENCE_LENGTH)

print('Preparing embedding matrix')
nb_words = min(MAX_NB_WORDS, len(word_index)) + 1
print('There are %d words' % nb_words)

# 把每个单词变成维度为EMBEDDING_DIM的向量,向量根据word2vec得到
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in Word2Vec:
        embedding_matrix[i] = Word2Vec[word]
# 查看哪些词没有词向量
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

生成embedding...
Found 6906 unique tokens
Preparing embedding matrix
There are 6907 words
Null word embeddings: 153


In [9]:
from ela_model import *
#from da_model import *
from keras.callbacks import EarlyStopping, ModelCheckpoint
print ("Running model")

# label = list(train_e.label)
# label.extend(list(train_s.label))
label = np.array(list(train.label))
# label = np.array(label)

folds = KFold(n_splits = 5, shuffle = False, random_state = 123)
pred_results = []
re_weight = True
val_loss_result = []
train_pred_results = []
for idx_train, idx_val in folds.split(data_1):
    data_1_train = data_1[idx_train]
    data_2_train = data_2[idx_train]
    #dis_train = distance_feature[idx_train]
    labels_train = label[idx_train]

    data_1_val = data_1[idx_val]
    data_2_val = data_2[idx_val]
    #dis_val = distance_feature[idx_val]
    labels_val = label[idx_val]
    
    Ela = ELA()
    model = Ela.build_model(embedding_matrix, data_1.shape[1])

    early_stopping = EarlyStopping(monitor='val_loss', patience=5)
    
    hist = model.fit([data_1_train, data_2_train],
                     labels_train, 
                     validation_data=([data_1_val, data_2_val], labels_val),
                     epochs=40,
                     batch_size=512,
                     shuffle=True, 
                     callbacks=[early_stopping],
                     verbose = 2)
    
    bst_val_score = min(hist.history['val_loss'])
    val_loss_result.append(bst_val_score)
    preds = model.predict([test_data_1, test_data_2], batch_size = 2048, verbose = 2)
    train_preds = model.predict([data_1_val, data_2_val], batch_size = 2048, verbose = 2)

    pred_results.append(preds)
    train_pred_results.extend(train_preds)

Running model
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 55)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 55)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 55, 300)      2072100     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 55, 600)      1442400     embedding_1[0][0]             

Train on 17120 samples, validate on 4280 samples
Epoch 1/40
 - 467s - loss: 0.6337 - acc: 0.6730 - val_loss: 0.5201 - val_acc: 0.7701
Epoch 2/40
 - 455s - loss: 0.5060 - acc: 0.7861 - val_loss: 1.1530 - val_acc: 0.4220
Epoch 3/40
 - 459s - loss: 0.4544 - acc: 0.8029 - val_loss: 0.4764 - val_acc: 0.7666
Epoch 4/40
 - 459s - loss: 0.3891 - acc: 0.8328 - val_loss: 0.4280 - val_acc: 0.8100
Epoch 5/40
 - 461s - loss: 0.3428 - acc: 0.8591 - val_loss: 0.3925 - val_acc: 0.8383
Epoch 6/40
 - 464s - loss: 0.3038 - acc: 0.8756 - val_loss: 0.3730 - val_acc: 0.8577
Epoch 7/40
 - 462s - loss: 0.2762 - acc: 0.8896 - val_loss: 0.3429 - val_acc: 0.8643
Epoch 8/40
 - 458s - loss: 0.2492 - acc: 0.9047 - val_loss: 0.3346 - val_acc: 0.8731
Epoch 9/40
 - 462s - loss: 0.2173 - acc: 0.9153 - val_loss: 0.3332 - val_acc: 0.8703
Epoch 10/40
 - 466s - loss: 0.1990 - acc: 0.9259 - val_loss: 0.3191 - val_acc: 0.8815
Epoch 11/40
 - 466s - loss: 0.1799 - acc: 0.9339 - val_loss: 0.5187 - val_acc: 0.7986
Epoch 12/40
 -

Train on 17120 samples, validate on 4280 samples
Epoch 1/40
 - 471s - loss: 0.6318 - acc: 0.6727 - val_loss: 0.6609 - val_acc: 0.7456
Epoch 2/40
 - 459s - loss: 0.5002 - acc: 0.7827 - val_loss: 0.5742 - val_acc: 0.6914
Epoch 3/40
 - 460s - loss: 0.4437 - acc: 0.8072 - val_loss: 0.4065 - val_acc: 0.8292
Epoch 4/40
 - 460s - loss: 0.3901 - acc: 0.8328 - val_loss: 0.4504 - val_acc: 0.7895
Epoch 5/40
 - 460s - loss: 0.3440 - acc: 0.8530 - val_loss: 0.3789 - val_acc: 0.8537
Epoch 6/40
 - 459s - loss: 0.3045 - acc: 0.8749 - val_loss: 0.3885 - val_acc: 0.8386
Epoch 7/40
 - 458s - loss: 0.2761 - acc: 0.8888 - val_loss: 0.3605 - val_acc: 0.8591
Epoch 8/40
 - 460s - loss: 0.2476 - acc: 0.9060 - val_loss: 0.4552 - val_acc: 0.8030
Epoch 9/40
 - 457s - loss: 0.2168 - acc: 0.9179 - val_loss: 0.3574 - val_acc: 0.8629
Epoch 10/40
 - 457s - loss: 0.2054 - acc: 0.9227 - val_loss: 0.3162 - val_acc: 0.8804
Epoch 11/40
 - 458s - loss: 0.1798 - acc: 0.9353 - val_loss: 0.3321 - val_acc: 0.8696
Epoch 12/40
 -

Train on 17120 samples, validate on 4280 samples
Epoch 1/40
 - 469s - loss: 0.6349 - acc: 0.6784 - val_loss: 0.6547 - val_acc: 0.7701
Epoch 2/40
 - 464s - loss: 0.5191 - acc: 0.7786 - val_loss: 0.5501 - val_acc: 0.7771
Epoch 3/40
 - 463s - loss: 0.4773 - acc: 0.7958 - val_loss: 0.4579 - val_acc: 0.7979
Epoch 4/40
 - 463s - loss: 0.4177 - acc: 0.8212 - val_loss: 0.4549 - val_acc: 0.8082
Epoch 5/40
 - 462s - loss: 0.3809 - acc: 0.8407 - val_loss: 0.4465 - val_acc: 0.8259
Epoch 6/40
 - 464s - loss: 0.3345 - acc: 0.8636 - val_loss: 0.3835 - val_acc: 0.8355
Epoch 7/40
 - 462s - loss: 0.3005 - acc: 0.8772 - val_loss: 0.3399 - val_acc: 0.8605
Epoch 8/40
 - 461s - loss: 0.2692 - acc: 0.8936 - val_loss: 0.3918 - val_acc: 0.8346
Epoch 9/40
 - 459s - loss: 0.2402 - acc: 0.9059 - val_loss: 0.5021 - val_acc: 0.7605
Epoch 10/40
 - 459s - loss: 0.2178 - acc: 0.9180 - val_loss: 0.3401 - val_acc: 0.8652
Epoch 11/40
 - 458s - loss: 0.1922 - acc: 0.9286 - val_loss: 0.3252 - val_acc: 0.8818
Epoch 12/40
 -

Train on 17120 samples, validate on 4280 samples
Epoch 1/40
 - 467s - loss: 0.6474 - acc: 0.6644 - val_loss: 0.5935 - val_acc: 0.7678
Epoch 2/40
 - 458s - loss: 0.5209 - acc: 0.7807 - val_loss: 0.7581 - val_acc: 0.5357
Epoch 3/40
 - 457s - loss: 0.4666 - acc: 0.7984 - val_loss: 0.4883 - val_acc: 0.7981
Epoch 4/40
 - 459s - loss: 0.4099 - acc: 0.8272 - val_loss: 0.5371 - val_acc: 0.7752
Epoch 5/40
 - 458s - loss: 0.3597 - acc: 0.8474 - val_loss: 0.3863 - val_acc: 0.8336
Epoch 6/40
 - 459s - loss: 0.3183 - acc: 0.8708 - val_loss: 0.4090 - val_acc: 0.8306
Epoch 7/40
 - 459s - loss: 0.2866 - acc: 0.8837 - val_loss: 0.4063 - val_acc: 0.8479
Epoch 8/40
 - 458s - loss: 0.2575 - acc: 0.8964 - val_loss: 0.7445 - val_acc: 0.6799
Epoch 9/40
 - 458s - loss: 0.2338 - acc: 0.9096 - val_loss: 0.3208 - val_acc: 0.8708
Epoch 10/40
 - 459s - loss: 0.2082 - acc: 0.9182 - val_loss: 0.5950 - val_acc: 0.7671
Epoch 11/40
 - 459s - loss: 0.1896 - acc: 0.9276 - val_loss: 0.3185 - val_acc: 0.8717
Epoch 12/40
 -

Train on 17120 samples, validate on 4280 samples
Epoch 1/40
 - 465s - loss: 0.6365 - acc: 0.6707 - val_loss: 0.4926 - val_acc: 0.8014
Epoch 2/40
 - 458s - loss: 0.5039 - acc: 0.7783 - val_loss: 0.5043 - val_acc: 0.7879
Epoch 3/40
 - 458s - loss: 0.4424 - acc: 0.8068 - val_loss: 0.4457 - val_acc: 0.8084
Epoch 4/40
 - 459s - loss: 0.3916 - acc: 0.8318 - val_loss: 0.3942 - val_acc: 0.8292
Epoch 5/40
 - 458s - loss: 0.3523 - acc: 0.8507 - val_loss: 0.5226 - val_acc: 0.7993
Epoch 6/40
 - 458s - loss: 0.3075 - acc: 0.8727 - val_loss: 0.3554 - val_acc: 0.8603
Epoch 7/40
 - 458s - loss: 0.2986 - acc: 0.8771 - val_loss: 0.4477 - val_acc: 0.7979
Epoch 8/40
 - 459s - loss: 0.2514 - acc: 0.9008 - val_loss: 0.4167 - val_acc: 0.8355
Epoch 9/40
 - 459s - loss: 0.2248 - acc: 0.9129 - val_loss: 0.3254 - val_acc: 0.8787
Epoch 10/40
 - 458s - loss: 0.2035 - acc: 0.9242 - val_loss: 0.4429 - val_acc: 0.8196
Epoch 11/40
 - 458s - loss: 0.1786 - acc: 0.9323 - val_loss: 0.5847 - val_acc: 0.7897
Epoch 12/40
 -

In [10]:
print (np.mean(val_loss_result))
res = (pred_results[0] + pred_results[1] + pred_results[2] +
     pred_results[3] + pred_results[4]) / 5
print (np.mean(res))

0.3208662311384611
0.37344524


In [16]:
len(train_pred_results)

31400

In [23]:
train_pred_results.head(30)

,0
0,5.527484e-07
1,4.526614e-01
2,5.396171e-02
3,9.952305e-01
4,1.263035e-02
5,1.226444e-04
6,7.148015e-03
7,1.306547e-01
8,1.365239e-02
9,3.593273e-03


In [12]:
train_pred_results.extend(res)

In [21]:
train_pred_results = pd.DataFrame(train_pred_results)
test['pred'] = res
test[['pred']].to_csv('ela_test_result.txt', index = None, header = None)
train_pred_results.to_csv('ela_result.dat',index = None)

In [16]:
len(res)

5000

In [17]:
len(res[np.where(res == 0)])

0

In [18]:
len(np.unique(test['spa_qura1']))

3153

In [19]:
len(np.unique(test['spa_qura2']))

2695

In [20]:
len(np.unique(train['spa_qura1']))

3521

In [21]:
len(np.unique(train['spa_qura2']))

5970

In [22]:
def aveResult(result_array_list):
    final = sum(result_array_list)/len(result_array_list)
    print(np.mean(final))
    final = pd.DataFrame(final)
    final.to_csv('result_.txt', index = None, header = None)

In [23]:
cv = [cv0,cv5]
aveResult(cv)

NameError: name 'cv0' is not defined

In [ ]:
xgb = pd.read_csv('result&gbdt.txt',header = None)
xgb.describe()

In [ ]:
np.mean(cv5)

In [ ]:
train = pd.read_csv('Are-you-happy/traditional_model/train.dat')
train.sentence_distance.head()

In [ ]:
df = pd.DataFrame()
df['sentence_distance'] = train['sentence_distance']
df['sentence_distance_tfidf'] = train['sentence_distance_tfidf']
df.to_csv('sentence_sim_feature.dat',index = False)

In [14]:
import pandas as pd
import numpy as np
#df = pd.read_csv('train.dat')
#gbdt = pd.read_csv('result/result_gbdt.txt',header = None)
cnn_ela_gbdt = pd.read_csv('result_gbdt.txt',header=None)
#gbdt_sim = pd.read_csv('result/result_gbdt&sim.txt',header=None)

In [ ]:
gbdt.describe()

In [17]:
cnn_ela_gbdt.mean()

0          0.240920
rescale    0.355809
dtype: float64

In [ ]:
gbdt_sim.mean()

In [3]:
def adj(x, te, tr):
        a = te / tr
        b = (1 - te) / (1 - tr)
        return a * x / (a * x + b * (1 - x))

In [16]:
#gbdt_sim['rescale'] = gbdt_sim.apply(lambda x:adj(x,0.3,0.25),axis=1)
cnn_ela_gbdt['rescale'] = cnn_ela_gbdt.apply(lambda x:adj(x[0],0.5,0.24),axis=1)

In [18]:
cnn_ela_gbdt['rescale'].to_csv('cnn_ela_da_gbdt_rescale_0.5.txt',index = False,header = None)

In [ ]:
cnn_gbdt['rescale'].to_csv('result_gbdt_cnn_rescale.txt',index = False,header = None)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
df = pd.read_csv('../diff_traditional/train.dat')
distance_feature = df[['len_char_s1','len_char_s2','fuzz_QRatio','fuzz_WRatio','sentence_distance_tfidf',
    'cosine_distance_ave_idf','skew_s1vec_ave_idf', 'skew_s2vec_ave_idf', 'kur_s1vec_ave_idf',
       'kur_s2vec_ave_idf']]
distance_feature = np.array(distance_feature)

In [ ]:
distance_feature = df[['len_char_s1','len_char_s2','fuzz_QRatio','fuzz_WRatio','sentence_distance_tfidf',
    'cosine_distance_ave_idf','skew_s1vec_ave_idf', 'skew_s2vec_ave_idf', 'kur_s1vec_ave_idf',
       'kur_s2vec_ave_idf']]

In [ ]:
distance_feature = np.array(distance_feature)
distance_feature[:2]

In [ ]:
data = pd.read_csv('cnn3_result.dat')
data.head(30)